# **Simple binary classifier** 

This notebook will generate a binary classifier that will identify if an attack is benign or not. The following algorithms will be used:
* Logistic regression 
* Random forest 
* CatBoost

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import imblearn
from collections import Counter

In [2]:
def get_data():
    data_path = os.path.dirname(os.path.abspath('')) + "/data/combined/"
    df = pd.read_pickle(data_path+'combined_cleaned.pickle')
    return df

def del_std_equal_to_zero(d):
    print(d.shape)
    counts = d.nunique()
    to_del = [i for i,v in enumerate(counts) if v == 1]
    print(d.columns[to_del])
    print(to_del)
    d = d.drop(d.columns[to_del], axis = 1)
    print(d.shape)
    return d



df = get_data() 
# df= df[:1000000]
X = df.drop(columns=['label', 'detect_type', 'detect_threat'])
y = df[['label', 'detect_type', 'detect_threat']]
X = del_std_equal_to_zero(X)
X = X.drop(columns=['timestamp', 'dst_port'])
# X = correlations(X)


# X_train_oh = one_hot_encoding(X, ['protocol'])




(16137183, 79)
Index(['bwd_psh_flags', 'bwd_urg_flags', 'fwd_byts_b_avg', 'fwd_pkts_b_avg',
       'fwd_blk_rate_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'bwd_blk_rate_avg'],
      dtype='object')
[33, 35, 57, 58, 59, 60, 61, 62]
(16137183, 71)


In [3]:
df.label.value_counts()

Benign                      13390249
DDOS attack-HOIC              686012
DDoS attacks-LOIC-HTTP        576191
DoS attacks-Hulk              461912
Bot                           286191
FTP-BruteForce                193354
SSH-Bruteforce                187589
Infilteration                 160639
DoS attacks-SlowHTTPTest      139890
DoS attacks-GoldenEye          41508
DoS attacks-Slowloris          10990
DDOS attack-LOIC-UDP            1730
Brute Force -Web                 611
Brute Force -XSS                 230
SQL Injection                     87
Name: label, dtype: int64

In [4]:
# class count
class_count_0, class_count_1 = df['detect_threat'].value_counts()

# divie class
class_0 = df[df['detect_threat'] == 0]
class_1 = df[df['detect_threat'] == 1]

In [5]:
# print the shape of the class
print('class 0:', class_0.shape)
print('\nclass 1:', class_1.shape)

class 0: (13390249, 82)

class 1: (2746934, 82)


# 1. Random under sampling

In [6]:
class_0_under = class_0.sample(class_count_1)

test_under = pd.concat([class_0_under, class_1], axis=0)

print("total class of 1 and 0:\n",test_under['detect_threat'].value_counts())

test_under['detect_threat'].value_counts().plot(kind='bar', title='Count (target)')
plt.show()

total class of 1 and 0:
 0    2746934
1    2746934
Name: detect_threat, dtype: int64


# 2. Random over sampling

In [7]:
class_1_over = class_1.sample(class_count_0, replace=True)

test_under = pd.concat([class_1_over, class_0], axis=0)

# print the number of class count
print('class count of 1 and 0:\n', test_under['detect_threat'].value_counts())

# plot the count
test_under['detect_threat'].value_counts().plot(kind='bar', title='Count (target)')
plt.show()

class count of 1 and 0:
 1    13390249
0    13390249
Name: detect_threat, dtype: int64


## Balance data with imbalance learn module

# 3. Random under-sampling with imblearn

In [8]:
# import library
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42, replacement=True)

# fit predictor and target varialbe
x_rus, y_rus = rus.fit_resample(X, y.detect_threat)

print('original dataset shape:', Counter(y.detect_threat))
print('Resample dataset shape', Counter(y_rus))

KeyboardInterrupt: 

# 4.Random oversampling with imblearn

In [ ]:
# import library
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target varaible
x_ros, y_ros = ros.fit_resample(X, y.detect_threat)

print('Original dataset shape', Counter(y.detect_threat))
print('Resample dataset shape', Counter(y_ros))

Original dataset shape Counter({0: 645573, 1: 354427})
Resample dataset shape Counter({0: 645573, 1: 645573})


# 5. Undersampling Tomek links

In [ ]:
# load library
from imblearn.under_sampling import TomekLinks

tl = TomekLinks(sampling_strategy='majority')

# fit predictor and target variable
x_tl, y_tl = tl.fit_resample(X, y.detect_threat)

print('Original dataset shape:', Counter(y.detect_threat))
print('Resample dataset shape:', Counter(y_tl))

# 6. Synthetic minority over-sampleing technique

In [ ]:
# load library
from imblearn.over_sampling import SMOTE

smote = SMOTE()

# fit target and predictor variable
x_smote , y_smote = smote.fit_resample(X, y.detect_threat)

print('Origianl dataset shape:', Counter(y.detect_threat))
print('Resampple dataset shape:', Counter(y_smote))

Origianl dataset shape: Counter({0: 645573, 1: 354427})
Resampple dataset shape: Counter({0: 645573, 1: 645573})


# 7. NearMiss

In [ ]:
from imblearn.under_sampling import NearMiss

nm = NearMiss()

x_nm, y_nm = nm.fit_resample(X, y.detect_threat)

print('Original dataset shape:', Counter(y.detect_threat))
print('Resample dataset shape:', Counter(y_nm))

Original dataset shape: Counter({0: 645573, 1: 354427})
Resample dataset shape: Counter({0: 354427, 1: 354427})
